In [ ]:
import torch
from torch import nn, Tensor
from tqdm import tqdm
from transformers import AutoTokenizer
import torch.nn.functional as F
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from metrics.perplexity import preprocess, calculate_perplexity
from metrics.lm_eval import eval_lambda

device = "cuda:7" if torch.cuda.is_available() else "cpu"

In [6]:
model_path = "/home/boboxa/llm_inference/llama3.2-3b"

model = AutoModelForCausalLM.from_pretrained(model_path).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_path)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="test")

data = preprocess(dataset['text'], tokenizer, device)

Token indices sequence length is longer than the specified maximum sequence length for this model (289077 > 131072). Running this sequence through the model will result in indexing errors


In [7]:
ppl_model = calculate_perplexity(model, data, tokenizer, model.device, 512)

100%|██████████| 564/564 [01:55<00:00,  4.88it/s]


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig

gptq_config = GPTQConfig(bits=4, dataset="c4", tokenizer=tokenizer)
quantized_model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto", quantization_config=gptq_config)

In [9]:
quantized_model.save_pretrained("/home/boboxa/llm_inference/llama3.2-3b-gptq")
tokenizer.save_pretrained("/home/boboxa/llm_inference/llama3.2-3b-gptq")

('/home/boboxa/llm_inference/llama3.2-3b-gptq/tokenizer_config.json',
 '/home/boboxa/llm_inference/llama3.2-3b-gptq/special_tokens_map.json',
 '/home/boboxa/llm_inference/llama3.2-3b-gptq/tokenizer.json')

In [ ]:
gptq_model = AutoModelForCausalLM.from_pretrained('/home/boboxa/llm_inference/llama3.2-3b-gptq', device_map=device)

In [30]:
ppl_model_gptq = calculate_perplexity(gptq_model, data, tokenizer, gptq_model.device, 512)

100%|██████████| 564/564 [00:48<00:00, 11.62it/s]


In [31]:
ppl_model_gptq

17.245698928833008

In [32]:
gptq_model = AutoModelForCausalLM.from_pretrained('shuyuej/Llama-3.2-3B-GPTQ', device_map=device)

In [33]:
ppl_model_gptq = calculate_perplexity(gptq_model, data, tokenizer, gptq_model.device, 512)

100%|██████████| 564/564 [00:48<00:00, 11.56it/s]


In [34]:
ppl_model_gptq

14.877720832824707

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig

gptq_config = GPTQConfig(bits=4, dataset="wikitext2", tokenizer=tokenizer)
quantized_model = AutoModelForCausalLM.from_pretrained(model_path, device_map=device, quantization_config=gptq_config)